In [ ]:
import torch
from fedgkt_trainers import GKTClientTrainer, GKTServerTrainer
from resnet_gkt_server import ResNet49
from resnet_gkt_client import ResNet8
from fedavg_utils import get_datasets, get_user_groups
import numpy as np
from reproducibility import make_it_reproducible
import pandas as pd

In [ ]:
make_it_reproducible(seed=0)

In [ ]:
client_number = 100
norm_type = 'Group Norm'
participation_frac = 0.1
iid = True
unbalanced = False

args_server = {
    'temperature': 3,
    'epochs_server': 10,
    'alpha': 0.5
}

args_client={
    'temperature': 3,
    'epochs_client': 1,
    'alpha': 0.5
}

communication_rounds = 10

In [ ]:
device = 'cuda' if torch.cuda.is_available else 'cpu'

In [ ]:
#SERVER
server_model = ResNet49(norm_type)
server_trainer = GKTServerTrainer(client_number, device, server_model, args_server)

In [ ]:
#CLIENT
client_model = ResNet8(norm_type)
trainset, testset = get_datasets()
user_groups = get_user_groups(trainset, iid=iid, unbalanced=unbalanced, tot_users=client_number)

clients = []
for client_idx in range(client_number):
  clients.append(GKTClientTrainer(client_idx, trainset, testset,
                                  user_groups[client_idx], device, client_model, args_client))

In [ ]:
for round in range(communication_rounds):
  print("Communication round: ", round+1)
  m = max(int(participation_frac*client_number), 1)
  chosen_users = np.random.choice(range(client_number), m, replace=False)
  print(f"Chosen users: {chosen_users}")
  for idx in chosen_users:
    extracted_features_dict, extracted_logits_dict, labels_dict,\
    extracted_features_dict_test, labels_dict_test = clients[idx].train()

    server_trainer.add_local_trained_result(idx, extracted_features_dict, extracted_logits_dict, labels_dict,\
    extracted_features_dict_test, labels_dict_test)

  server_trainer.train(round)

  for idx in chosen_users:
    global_logits = server_trainer.get_global_logits(idx)
    clients[idx].update_large_model_logits(global_logits)
  server_trainer.remove_records()

train_metrics, test_metrics = server_trainer.get_metrics_lists()

In [ ]:
df = pd.DataFrame(train_metrics)
df.to_csv(f"train_{norm_type}_{'iid' if iid else 'noniid'}_{'unbalanced' if unbalanced else 'balanced'}.csv", index=False)

In [ ]:
df = pd.DataFrame(test_metrics)
df.to_csv(f"test_{norm_type}_{'iid' if iid else 'noniid'}_{'unbalanced' if unbalanced else 'balanced'}.csv", index=False)